In [16]:
using JuMP, Gurobi, XLSX

In [17]:
xf = XLSX.readxlsx("toy-stores.xlsx")

XLSXFile("toy-stores.xlsx") containing 1 Worksheet
            sheetname size          range        
-------------------------------------------------
               Sheet1 102x12        A1:L102      


In [18]:
data = xf["Sheet1"]

102×12 XLSX.Worksheet: ["Sheet1"](A1:L102)

In [19]:
model = Model(with_optimizer(Gurobi.Optimizer))
M = 100
N = 24

Academic license - for non-commercial use only


24

In [20]:
set_optimizer_attribute(model, "Presolve", 0)
set_optimizer_attribute(model, "Heuristics", 0)
set_optimizer_attribute(model, "Cuts", 0)

0

In [21]:
# Variables
@variable(model, 0 <= x[1:M,1:N]  <= 1) # + Constraint (4)
@variable(model, 0 <= y[1:N], Bin ) # + Constraint (5)

24-element Array{VariableRef,1}:
 y[1]
 y[2]
 y[3]
 y[4]
 y[5]
 y[6]
 y[7]
 y[8]
 y[9]
 y[10]
 y[11]
 y[12]
 y[13]
 y[14]
 y[15]
 y[16]
 y[17]
 y[18]
 y[19]
 y[20]
 y[21]
 y[22]
 y[23]
 y[24]

In [31]:
# Compute costs C_ij and fixed costs f_j
C = zeros(Float64, (M,N))
f = zeros(Float64,N)
for j in 1:N
    lat2 = deg2rad(data[j+2,11])
    long2 = deg2rad(data[j+2,10])
    f[j]=data[j+2,12]
    for i in 1:M
        lat = deg2rad(data[i+2,4]) 
        long = deg2rad(data[i+2,3])
        if lat==lat2 && long ==long2 
            C[i,j]=0
        else
            C[i,j]= acos(sin(lat)*sin(lat2)+cos(lat)*cos(lat2)*cos(long-long2))*3958.8
        end
    end
end


In [33]:
# Objective function
@objective(model, Min, sum(C .* x) + sum(f .* y))

1171.975775352057 x[1,1] + 1382.1777666600115 x[3,1] + 994.4825123333845 x[4,1] + 350.00474780722516 x[5,1] + 412.74032090472724 x[6,1] + 968.1832949652545 x[7,1] + 1410.1397371346568 x[8,1] + 863.5199072096237 x[9,1] + 961.3813008419755 x[10,1] + 1720.2313062781707 x[11,1] + 1600.274212383271 x[12,1] + 1336.0302381301865 x[13,1] + 1182.5441998004792 x[14,1] + 1426.4647440279866 x[15,1] + 889.2720558342551 x[16,1] + 989.4493937030827 x[17,1] + 1170.9531390120937 x[18,1] + 300.38234733273003 x[19,1] + 333.7895151446895 x[20,1] + 1014.4404496281066 x[21,1] + 357.00552079538824 x[22,1] + 1285.041542255944 x[23,1] + 813.9861982481183 x[24,1] + 1108.4407272519593 x[25,1] + 1428.0889805145046 x[26,1] + 990.8970570063148 x[27,1] + 923.4053254134172 x[28,1] + 602.1522424305485 x[29,1] + 1016.5563978972305 x[30,1] + 1233.7346579419473 x[31,1] + 1425.8563175133 x[32,1] + 1630.8790214516882 x[33,1] + 1429.9369718092344 x[34,1] + 1430.8036309692106 x[35,1] + 530.3528906118232 x[36,1] + 927.2841163

In [49]:
# Constraint (2)
@constraint(model, [i = 1:M], sum(x, dims=2) .== ones(Float64, M))
#sum(x,dims=2)
#@constraint(model, [j = 1:N], sum(x[:,j]) == 1)

100-element Array{Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.EqualTo{Float64}},ScalarShape},2},1}:
 [x[1,1] + x[1,2] + x[1,3] + x[1,4] + x[1,5] + x[1,6] + x[1,7] + x[1,8] + x[1,9] + x[1,10] + x[1,11] + x[1,12] + x[1,13] + x[1,14] + x[1,15] + x[1,16] + x[1,17] + x[1,18] + x[1,19] + x[1,20] + x[1,21] + x[1,22] + x[1,23] + x[1,24] = 1.0; x[2,1] + x[2,2] + x[2,3] + x[2,4] + x[2,5] + x[2,6] + x[2,7] + x[2,8] + x[2,9] + x[2,10] + x[2,11] + x[2,12] + x[2,13] + x[2,14] + x[2,15] + x[2,16] + x[2,17] + x[2,18] + x[2,19] + x[2,20] + x[2,21] + x[2,22] + x[2,23] + x[2,24] = 1.0; … ; x[99,1] + x[99,2] + x[99,3] + x[99,4] + x[99,5] + x[99,6] + x[99,7] + x[99,8] + x[99,9] + x[99,10] + x[99,11] + x[99,12] + x[99,13] + x[99,14] + x[99,15] + x[99,16] + x[99,17] + x[99,18] + x[99,19] + x[99,20] + x[99,21] + x[99,22] + x[99,23] + x[99,24] = 1.0; x[100,1] + x[100,2] + x[100,3] + x[100,4] + x[100,5] + x[100,6] + x[100,7] + x[100,

In [51]:
# Constraint (3)
@constraint(model, [i = 1:M], x[i,:] .<= y)

100-element Array{Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1},1}:
 [x[1,1] - y[1] ≤ 0.0, x[1,2] - y[2] ≤ 0.0, x[1,3] - y[3] ≤ 0.0, x[1,4] - y[4] ≤ 0.0, x[1,5] - y[5] ≤ 0.0, x[1,6] - y[6] ≤ 0.0, x[1,7] - y[7] ≤ 0.0, x[1,8] - y[8] ≤ 0.0, x[1,9] - y[9] ≤ 0.0, x[1,10] - y[10] ≤ 0.0  …  x[1,15] - y[15] ≤ 0.0, x[1,16] - y[16] ≤ 0.0, x[1,17] - y[17] ≤ 0.0, x[1,18] - y[18] ≤ 0.0, x[1,19] - y[19] ≤ 0.0, x[1,20] - y[20] ≤ 0.0, x[1,21] - y[21] ≤ 0.0, x[1,22] - y[22] ≤ 0.0, x[1,23] - y[23] ≤ 0.0, x[1,24] - y[24] ≤ 0.0]
 [x[2,1] - y[1] ≤ 0.0, x[2,2] - y[2] ≤ 0.0, x[2,3] - y[3] ≤ 0.0, x[2,4] - y[4] ≤ 0.0, x[2,5] - y[5] ≤ 0.0, x[2,6] - y[6] ≤ 0.0, x[2,7] - y[7] ≤ 0.0, x[2,8] - y[8] ≤ 0.0, x[2,9] - y[9] ≤ 0.0, x[2,10] - y[10] ≤ 0.0  …  x[2,15] - y[15] ≤ 0.0, x[2,16] - y[16] ≤ 0.0, x[2,17] - y[17] ≤ 0.0, x[2,18] - y[18] ≤ 0.0, x[2,19] - y[19] ≤ 0.0, x[2,20] - y[20] ≤ 0.0, x[2,21] - y[21] ≤ 

In [ ]:
# Constraint (6)
@constraint(model, sum(x, dims=1) .<= M * transpose(y))